<a href="https://colab.research.google.com/github/jepilogo97/nlp/blob/main/nlp-with-gpt/nlp_with_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot - RAG

##### Jean Pierre Londoño González
##### Mini-Proyecto de chatbot utilizando RAG
##### 28SEP2025

Partiendo de los conceptos establecidos en el [1-ollama-rag.ipynb](./1-ollama-rag.ipynb) ahora exploraremos una implementación con herramientas más maduras que facilitan el manejo de RAGs y LLMs. Además de eso, continuaremos el uso de [Ollama](https://ollama.com) y añadiremos el uso de [LangChain](https://www.langchain.com) un framework orientado al desarrollo de LLMs y agentes de IA. El objetivo final es el mismo, crear un RAG a partir de los documentos de wikihow. Observaremos que algunos de los pasos del notebook anterior se hacen de forma explicita.

**Nota:** En este notebook no se realizará el entrenamiento de ningún modelo, se utilizarán modelos pre-entrenados y se combinaran de una forma interesante para hacer uso de cada componente a su modo y crear un agente de conversación

#### Referencias
- Dataset: https://huggingface.co/datasets/RamAnanth1/lex-fridman-podcasts
- https://huggingface.co/EleutherAI/gpt-neo-125m

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la manipulación de datos y la construcción del modelo. Esto incluye NumPy y pandas para el manejo y análisis de datos; Hugging Face Datasets y Transformers para la carga de corpus y la tokenización; y PyTorch junto con PyTorch Lightning para definir, entrenar y evaluar el modelo de manera estructurada.

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

C:\Users\jepil\AppData\Local\Temp\ipykernel_7308\2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [1]:
!wget -O requirements.txt https://raw.githubusercontent.com/jepilogo97/nlp/main/chatbot-rag/requirements.txt
!pip install -r requirements.txt

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [6]:
# Procesamiento de lenguaje natural y utilidades
import numpy as np  # Cálculo numérico y manejo de arreglos multidimensionales
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame

pd.set_option("display.max_rows", None)     # Todas las filas
pd.set_option("display.max_columns", None)  # Todas las columnas
pd.set_option("display.width", None)        # No cortar líneas

from datasets import Dataset, load_dataset, concatenate_datasets  # Carga y combinación de datasets de Hugging Face
from datasets.dataset_dict import DatasetDict
from collections import Counter  # Conteo de frecuencias de elementos (tokens, palabras, etc.)

import matplotlib.pyplot as plt
# Deep Learning con PyTorch
import torch  # Librería principal de tensores y operaciones en GPU/CPU
import torch.nn as nn  # Definición de capas y módulos de redes neuronales
import torch.nn.functional as F  # Funciones de activación y operaciones matemáticas de redes
from torch.utils.data import random_split, DataLoader, Subset  # Utilidades para crear y dividir datasets, cargar lotes y trabajar con subconjuntos
from torchinfo import summary

# Entrenamiento estructurado con PyTorch Lightning
from pytorch_lightning import LightningModule, Trainer  # Clase base y manejador de entrenamiento de modelos
from pytorch_lightning.loggers import TensorBoardLogger  # Registro de métricas e historial en TensorBoard
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint  # Detener entrenamiento si no mejora la métrica
from torchmetrics import Accuracy  # Métrica de precisión para clasificación supervisada

# Tipado para mayor legibilidad y validación de funciones
from typing import Optional, Tuple # Definición de tipos de datos para funciones y estructuras

from tqdm.auto import tqdm  # Barra de progreso adaptable para bucles
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments  # Tokenizador automático de modelos preentrenados de Hugging Face
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode  # Conversión de bytes a caracteres Unicode (usado en tokenización tipo GPT-2)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase


# Métricas de evaluación con Scikit-learn
from sklearn.model_selection import train_test_split  # División de datos en conjuntos de entrenamiento y prueba
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas de evaluación de modelos de clasificación

ImportError: DLL load failed while importing lib: No se encontró el proceso especificado.

### 2. Exploración de dataset

Se utilizará el dataset RagQuAS, el cuál contiene ejemplos en una gran cantidad de dominios: Hobbies, Lingüística, Mascotas, Salud, astronomía, atención al cliente, coches, cotidiano, documentación, energía, esquí, estafas, gastronomía, hobbies, idiomas, juegos, lenguaje, manicura, música, patinaje, primeros auxilios, receta, reciclaje, reclamaciones, seguros, tenis, transporte, turismo, veterinaria, viajes, yoga.

- https://huggingface.co/datasets/IIC/RagQuAS

In [ ]:
dataset = load_dataset('IIC/RagQuAS')
dataset

In [ ]:
pd.set_option('display.max_colwidth', 100)
dataset.set_format(type='pandas')
df = dataset.to_pandas()
df.head(15)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
df.document.apply(lambda text: len(text.split(' '))).hist(ax=ax)
ax.set_title("Distribución por número de palabras por documento")
ax.set_xlabel("Número de palabras")
ax.set_ylabel("Frecuencia")
plt.show()

In [ ]:
limit = 5000
documents = dataset.take(limit).map(lambda example: {"url": example['url'], "title": example['title'], "text": example['title'] + '\n' + example['summary'] + '\n' + example['document']})
documents

### 2. Implementando una función de generación

Ahora, la idea es que este modelo nos sirva para generar texto de forma recurrente e incremental. En la última capa de los modelos tipo GPT encontrarémos un tensor con forma $(b, s, v)$, donde:

- $b$: Es el tamaño del bache, o la cantidad de secuencias a procesar.
- $s$: Es la longitud de la secuencia de entrada.
- $v$: Es el tamaño del vocabulario del modelo, cuantos tokens soporta.

Pero este es el tensor de salida, por qué tiene la forma de la secuencia de entrada?, porque cada posición en la salida corresponde a la la predicción del siguiente token de esa posición en la secuencia de entrada. En otras palabras, lo que obtenemos como predicción, es una secuencia de igual tamaño a la de entrada, movida un token hacia adelante, lo que efectivamente nos predice un solo token a la vez y por ende, el token que nos insteresa, es el último.

Lo que obtenemos en este tensor es además los logits de TODO el vocabulario del modelo, con los cuales podemos calcular las probabilidades de que cada uno sea el que continue en la secuencia. Hay varias formas de decodificar el siguiente token, la más fácil de implementar sería una decodificación codiciosa (greedy) del siguiente token, que consiste simplemente en seleccionar el token con la probabilidad más alta. Este es un enfoque simple y efectivo para algunos casos, pero al mismo tiempo sufre de poca variabilidad e incluso puede caer en generación repetitiva.

Otra opción es el muestreo, ya que justamente podemos obtener probabilidades del siguiente token, lo más lógico sería muestrear con esas opciones de probabilidad, de este modo podemos obtener mayor diversidad a la hora de generar el texto, al costo eso si de que haya mayor aleatoridad ya que se le daría la oportunidad a incluso tokens con baja probabilidad, de ser seleccionados.

Otra opción podría ser un balanceo entre una decodificación greedy y una por muestreo, en función de otro hiperparámetro que podemos definir. Esta sería una técnica muy común en el contexto de Reinforcement Learning llamade e-greedy. Se hace la aclaración de que en este ejemplo no harémos nada de RL, solamente se hace mención de esta técnica para balancear entre explotación y exploración.

In [ ]:
def generate(
        model: nn.Module,
        tokenizer: PreTrainedTokenizerBase,
        start: str,
        max_length: int = 1000,
        eps: float = 0.5,
        top_n: int = 5,
        return_iterations: bool = False,
        device: str = "cpu") -> Tuple[str, Optional[pd.DataFrame]]:

    output = [start]
    iterations = []
    with torch.no_grad():
        input_ids = tokenizer(output[-1], return_tensors='pt')['input_ids'].to(device)
        for _ in range(max_length):
            # Tomamos los logits producidos por la última capa del modelo
            # Estos corresponden al siguiente token por cada posición de la cadena
            logits = model(input_ids=input_ids).logits
            # Por lo tanto, el que nos interesa es el último, que correspondería a la
            # predicción del siguiente token después del final de la cadena original
            # A este aplicamos un softmax para obtener las probabilidades por cada
            # token del vocabulario para estar presente en la cadena.
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Simplemente ordenamos por probabilidad de forma descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Utilizamos una politica tipo e-greedy para obtener el siguiente token de la secuencia
            # Un eps>=1 quiere decir que siempre se va seleccionar el token de forma 'greedy', es decir
            # siempre se toma el token con probabilidad más alta.

            # Un eps=0 quiere decir que siempre se va a muestrear el siguiente token en función
            # de las probabilidades de cada token

            # Un 0<eps<1 va a balancear de forma binomial entre tomar el token con la
            # probabilidad más alta y muestrear el token en función de sus probabilidades.
            if np.random.random_sample(1)[0] < eps:
                # Se toma el mejor token
                next_token = sorted_tokens[0].unsqueeze(dim=0)
            else:
                # Se muetrea el token de la probabilidad de distribución
                next_token = torch.multinomial(probs, 1)

            if return_iterations:
                # Mantenemos pista de todas las iteraciones para análisis
                iteration = {'input': ''.join(output)}
                best_n = sorted_tokens[:top_n].cpu().tolist()
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})' for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            output.append(tokenizer.decode(next_token))
            input_ids = torch.cat([input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        output_text = ''.join(output)
        if not return_iterations:
            return output_text, None
        else:
            df = pd.DataFrame(iterations)
            return output_text, df

Ahora observemos que pasa cuando generamos texto con nuestra función y algunos parámetros.

Primero, observemos que pasa cuando pasamos un `eps=1` que quiere decir que la generación va a ser de tipo greedy:

In [ ]:
output_text, iterations_df = generate(model, tokenizer, text, max_length=15, eps=1.0, top_n=10, return_iterations=True, device=device)
print(output_text)
iterations_df.head(15)

Observamos como el input progresa a la vez que las opciones de tokens que hay. Sin importar cuantas veces invoquemos a la función con los mismos parámetros, siempre vamos a obtener los mismos resultados.

Ahora, observemos que pasa si introducimos exploración al reducir el `eps=0.5`, lo cual nos dice que aproximadamente la mitad de las veces va a elegir el siguiente token muestreando y la otra mitad explotando.

In [ ]:
output_text, _ = generate(model, tokenizer, text, max_length=100, eps=0.5, device=device)
print(output_text)

Lo primero observado es que aparecen extrañamente muchos signos de interrogación dentro del texto generado.
Al intentar aumentar el max length se evidencia que es demasiado costoso, pasar de 100 a 1000 toma un tiempo bastante considerable.

### 3. Generando texto con las utilidades del modelo

Ahora, la clase de Huggingface implementa la función `generate` que hace la labor de generación por nosotros, incluyendo las opciones de muestreo y explotación como hemos observado. Solo que además permite otra serie de parámetros y opciones para controlar la generación de texto.

In [ ]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=100, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output[0]))

La exploración de la temperatura promueve creatividad y diversidad, pero con casos que evidencian incoherencia o “alucinaciones”.

### 4. Carga de dataset

Ahora, intentemos hacer fine tuning a nuestro modelo:

In [ ]:
dataset = load_dataset("RamAnanth1/lex-fridman-podcasts")
dataset

In [ ]:
dataset['train'][0]

In [ ]:
dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df.head(10)

In [ ]:
df['Palabras por podcast'] = df['captions'].str.split().apply(len)
df['Palabras por podcast'].median()

### 5. Fine tuning

Aquí podemos observar que la mediana de longitud en terminos de palabras es de 20371. Esto es esperado, pues los podcast son largos por naturaleza.

Ahora, prepararémos el conjunto de datos para entrenamiento.

In [ ]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(examples['captions'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function

Los modelos GPT no esperan otra cosa más que los `input_ids`, por lo que retirarémos todas las demás columnas del dataset ya que no nos son de utilidad en este momento.

In [ ]:
dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')
tokenized_dataset

Finalmente procedemos a definir el entrenamiento.

In [ ]:
batch_size = 2 if IN_COLAB else 1
logging_steps = len(tokenized_dataset['train']) // batch_size
# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps,
    report_to='none'
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

In [ ]:
%%time
trainer.train()

### 6. Resultados

Ahora observemos los resultados.

In [ ]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=2000, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

Tras generar un texto de 2 000 tokens, se identificaron algunas inconsistencias, aunque el resultado es aceptable para el tamaño del modelo. No obstante, a partir de cierto punto el modelo entra en un bucle repitiendo la frase “the mind”, lo que podría deberse a patrones presentes en los datos de entrenamiento o a una configuración de decodificación poco adecuada. Este comportamiento pone de manifiesto la fuerte influencia que ejerce el conjunto de datos de entrenamiento en el desempeño final, un aspecto crucial a considerar al utilizar o realizar fine-tuning de modelos de lenguaje.

### 7. Conclusiones

#### Eficacia del flujo de análisis

- Al comparar arquitecturas de modelos de lenguaje como BERT y GPT, se identifican similitudes en su capacidad de generar representaciones ricas del lenguaje, algunas diferencias clave en su estructura y en su proceso de entrenamiento.

- Ambos modelos demuestran que un pre-entrenamiento sólido y la construcción de embeddings de alta calidad son factores críticos para alcanzar buenos resultados en tareas posteriores, evitando costos de entrenamiento desde cero.

#### Rendimiento del modelo

- Tanto BERT como GPT pueden adaptarse a una amplia variedad de tareas posteriores (clasificación, generación de texto, análisis semántico, etc.), lo que valida la versatilidad de los enfoques de transfer learning y fine tuning.

- La elección del modelo y la estrategia de entrenamiento depende de la tarea: BERT sobresale en comprensión y análisis de contexto bidireccional, mientras que GPT destaca en generación de texto coherente y fluido.

#### Limitaciones observadas

- Los modelos generativos enfrentan un dilema de exploración–explotación: una decodificación enfocada en la explotación (p. ej. greedy search) brinda mayor precisión pero tiende a producir textos monótonos; en cambio, la exploración (p. ej. sampling con temperatura alta) promueve creatividad y diversidad, pero con riesgo de incoherencia o “alucinaciones”.

- La calidad del modelo depende en gran medida de los datos de entrenamiento. Conjuntos de datos sesgados, poco representativos o de baja calidad pueden degradar el desempeño e introducir sesgos o errores difíciles de corregir.

#### Áreas de mejora

- Profundizar en la selección y curaduría de datasets, asegurando diversidad, equilibrio y relevancia para el dominio de aplicación.

- Experimentar con estrategias de decodificación y con la afinación de hiperparámetros para encontrar el punto óptimo entre creatividad, coherencia y precisión.

- Explorar técnicas de optimización y compresión que permitan desplegar modelos grandes en entornos de recursos limitados.

#### Valor práctico

- La adopción de modelos pre-entrenados como GPT ofrece una base sólida y flexible para proyectos de NLP, equilibrando costo, tiempo y calidad.

- El entendimiento de los trade-offs entre exploración y explotación, así como la adecuada selección de datos y métodos de decodificación, es esencial para alinear el modelo con los objetivos específicos del negocio y minimizar riesgos de sesgos o resultados no deseados.

### 8. Apendice

In [ ]:
import pkg_resources

libs = [
    "numpy",
    "pandas",
    "datasets",
    "torch",
    "pytorch-lightning",
    "torchmetrics",
    "tqdm",
    "transformers",
    "scikit-learn"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except Exception:
        print(f"{lib}")

In [ ]:
 ## Solo correr en local

# import nbformat

## Cargar notebook
# with open("nlp_with_gpt.ipynb", "r", encoding="utf-8") as f:
    # nb = nbformat.read(f, as_version=4)

## Eliminar widgets corruptos si existen
# if "widgets" in nb["metadata"]:
    # del nb["metadata"]["widgets"]

## Guardar reparado
# with open("nlp_with_gpt.ipynb", "w", encoding="utf-8") as f:
    # nbformat.write(nb, f)